## Problem 04
Find the solution to: <br>
\begin{equation}
\begin{aligned}
& \underset{X}{\text{min}}
& & f = x_1^2 + x_2^2 + x^2_3 \\
& \text{subject to}
& & h_1 = \frac{x_1^2}{4} + \frac{x_2^2}{5} + \frac{x_3^2}{25} - 1 = 0 \\
& & & h_2 = x_1 + x_2 - x_3 = 0
\end{aligned}
\end{equation}

With two contraints and three variables, there will be two state variables and one driving variable. I will make $x_1\ $ my driving variable and $x_2\ $ and $x_3\ $ my state variables

In [28]:
import numpy as np
x1_0 = 1                # initial driving condition
x2_0 = 1.561370113      # initial state conditions
x3_0 = 2.561370113
X = np.array([x2_0, x3_0, x1_0])
k = 0                   # initialize counter for solver
eps = 10**-3            # minimum error for slope
sk = np.array([X[0], X[1]])     # state variables
dk = np.array([X[2]])           # driving variable
dfdx1 = np.array([2 * X[2]])    # dfdd (partial)
dfds  = np.array([2 * X[0], 2 * X[1]]) # dfds (partial)
dhds  = np.array([[2 / 5 * X[0], 2 / 25 * X[1]],
                  [1, -1]])
dhdd  = np.array([[1 / 2 * X[2]], [1]])
dfdd  = dfdx1 - np.matmul(np.matmul(dfds, np.linalg.inv(dhds)), dhdd) # non-linear dfdd

def minfunc(X):  # calculation of function to be minimized
    x1 = X[2]
    x2 = X[0]
    x3 = X[1]
    return x1**2 + x2**2 + x3**2

def confunc(X): # claclulation of constraint conditions
    x1 = X[2]
    x2 = X[0]
    x3 = X[1]
    h1 = x1**2 / 4 + x2**2 / 5 + x3**2 /25 - 1
    h2 = x1 + x2 -x3
    return np.array([[h1], [h2]])

def linesearch(dfdd, sk, dk, dhds_line, dhdd_line):  #inexact line search
    alpha = 1
    b = 0.3
    t = 0.1
    counter = 0
    dhds_inv = np.linalg.inv(dhds_line)  # calculates inverse of dhds for function evaluation
    skstep_matrix = np.transpose(np.matmul(np.matmul(dhds_inv, dhdd_line), dfdd))
    sk1 = sk + alpha * skstep_matrix  # state step
    dk1 = dk - alpha * dfdd # driving step
    X_step = np.concatenate((sk1, dk1), axis=None)
    f = minfunc(X_step) # function analysis
    phi = minfunc(np.concatenate((sk, dk), axis=None)) - alpha * t * (dfdd)**2 #phi analysis with step
    while f > phi and counter<50:
        alpha = b * alpha
        sk1 = sk + alpha * skstep_matrix
        dk1 = dk - alpha * dfdd
        X_step = np.concatenate((sk1, dk1), axis=None)
        f = minfunc(X_step)
        phi = minfunc(np.concatenate((sk, dk), axis=None)) - alpha * t * (dfdd)**2
        counter += 1
    if counter == 50:
        print("linesearch failed to converge")
    return alpha, skstep_matrix

def solvefunc(driving, state_approx):  # Newton rhobson method for finding new state values
    c = 0
    x1 = driving
    x2 = state_approx[0]
    x3 = state_approx[1]
    skactual = np.array([x2, x3])
    Xpart = np.concatenate((state_approx, driving), axis=None)
    h = confunc(Xpart)
    while np.linalg.norm(h) > 10**-3 and c<30:
        sktran = np.array([[x2], [x3]]) - np.matmul(np.linalg.inv(np.array([[2 / 5 * Xpart[0], 2 / 25 * Xpart[1]], [1, -1]])), confunc(Xpart))
        x2 = sktran[0]
        x3 = sktran[1]
        skactual = np.concatenate((x2, x3), axis=None)
        Xpart = np.concatenate((x2, x3, x1), axis=None)
        h = confunc(Xpart)
        c += 1
    if c == 30:
        print("Newton Method did not Converge")
    return skactual

while dfdd**2 > eps and k < 30:  # solver loop
    alphak, stepmatrix= linesearch(dfdd, sk, dk, dhds, dhdd)
    dk = dk - alphak * dfdd
    sk_lin_approx = sk + alphak * stepmatrix
    sk = solvefunc(dk, sk_lin_approx)
    X = np.concatenate((sk, dk), axis=None)
    dfdx1 = np.array([2 * X[2]])
    dfds  = np.array([2 * X[0], 2 * X[1]])
    dhds  = np.array([[2 / 5 * X[0], 2 / 25 * X[1]],
                      [1, -1]])
    dhdd  = np.array([[1 / 2 * X[2]], [1]])
    dfdd  = dfdx1 - np.matmul(np.matmul(dfds, np.linalg.inv(dhds)), dhdd)
    k += 1
    print(X, dfdd)

print("x1 = " + np.array2string(X[2]) + "\nx2 = " + np.array2string(X[0]) + "\nx3 = " + np.array2string(X[1]) + "\ndfdd = " + np.array2string(dfdd))

[1.56401733 2.56090239 0.99688506] [0.12896516]
[1.57384693 2.55912512 0.9852782 ] [0.17926598]
[1.58733336 2.55647762 0.96914426] [0.24808734]
[1.6457661  2.54048416 0.89471805] [0.54789197]
[1.75898312 2.48933359 0.73035046] [1.12794199]
[1.93137415 2.32334202 0.39196787] [2.04030253]
Newton Method did not Converge
[ 7.20246076e+02  7.56674200e+02  7.20025953e+02  7.56454077e+02
 -2.20122891e-01] [-609.74713499]


ValueError: operands could not be broadcast together with shapes (4,) (2,) 